In [ ]:
class SegmentTree:
    def __init__(self, data, merge): 
        '''
        data:传入的数组
        merge:处理的业务逻辑，例如求和/最大值/最小值，lambda表达式
        '''
        self.data = data
        self.n = len(data) # 申请4倍data长度的空间来存线段树节点
        self.tree = [None] * (4 * self.n) # 索引i的左孩子索引为2i+1，右孩子为2i+2
        self._merge = merge
        if self.n:
            self._build(0, 0, self.n-1)

    def query(self, ql, qr):
        '''
        返回区间[ql,..,qr]的值
        '''
        return self._query(0, 0, self.n-1, ql, qr)

    def update(self, index, value):
        # 将data数组index位置的值更新为value,然后递归更新线段树中被影响的各节点的值
        self.data[index] = value
        self._update(0, 0, self.n-1, index)

    def _build(self, tree_index, l, r):
        '''
        递归创建线段树
        tree_index : 线段树节点在数组中位置
        l, r : 该节点表示的区间的左,右边界
        '''
        if l == r:
            self.tree[tree_index] = self.data[l]
            return
        mid = (l+r) // 2 # 区间中点,对应左孩子区间结束,右孩子区间开头
        left, right = 2 * tree_index + 1, 2 * tree_index + 2 # tree_index的左右子树索引
        self._build(left, l, mid)
        self._build(right, mid+1, r)
        self.tree[tree_index] = self._merge(self.tree[left], self.tree[right])

    def _query(self, tree_index, l, r, ql, qr):
        '''
        递归查询区间[ql,..,qr]的值
        tree_index : 某个根节点的索引
        l, r : 该节点表示的区间的左右边界
        ql, qr: 待查询区间的左右边界
        '''
        if l == ql and r == qr:
            return self.tree[tree_index]

        mid = (l+r) // 2 # 区间中点,对应左孩子区间结束,右孩子区间开头
        left, right = tree_index * 2 + 1, tree_index * 2 + 2
        if qr <= mid:
            # 查询区间全在左子树
            return self._query(left, l, mid, ql, qr)
        elif ql > mid:
            # 查询区间全在右子树
            return self._query(right, mid+1, r, ql, qr)

        # 查询区间一部分在左子树一部分在右子树
        return self._merge(self._query(left, l, mid, ql, mid), 
                          self._query(right, mid+1, r, mid+1, qr))

    def _update(self, tree_index, l, r, index):
        '''
        tree_index:某个根节点索引
        l, r : 此根节点代表区间的左右边界
        index : 更新的值的索引
        '''
        if l == r == index:
            self.tree[tree_index] = self.data[index]
            return
        mid = (l+r)//2
        left, right = 2 * tree_index + 1, 2 * tree_index + 2
        if index > mid:
            # 要更新的区间在右子树
            self._update(right, mid+1, r, index)
        else:
            # 要更新的区间在左子树index<=mid
            self._update(left, l, mid, index)
        # 里面的小区间变化了，包裹的大区间也要更新
        self.tree[tree_index] = self._merge(self.tree[left], self.tree[right])

class NumArray:
    def __init__(self, nums: List[int]):
        self.segment_tree = SegmentTree(nums, lambda x, y : x + y)
        
    def update(self, i: int, val: int) -> None:
        self.segment_tree.update(i, val)
        
    def sumRange(self, i: int, j: int) -> int:
        return self.segment_tree.query(i, j)
        
# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(i,val)
# param_2 = obj.sumRange(i,j)

In [47]:
from typing import List

# segment tree + DFS
class SegmentTree:
    def __init__(self, values):
        self.tree = {}
        self.values = values
        n = len(values)
        self.build(0, n-1, 0)
    
    def build(self, start, end, node):
        # 构建树
        if start == end:
             # 存储了最小值以及最小值在 values中的下标
            self.tree[node] = (self.values[start], start)
            return
        mid = (start + end) // 2
        left_node = 2 * node + 1
        right_node = 2 * node + 2
        self.build(start, mid, left_node)
        self.build(mid+1, end, right_node)
        # 当前节点存储了左子树与右子树之间的最小值与 index
        if self.tree[left_node][0] < self.tree[right_node][0]:
            self.tree[node] = self.tree[left_node]
        else:
            self.tree[node] = self.tree[right_node]
    
    def queryMinVal(self, start, end, node, left, right):
        # 查找在 values 中 left 和 right 区间内的最小值
        if left > end or right < start:
            return float('inf'), None
        elif start == end or (start >= left and end <= right):
            return self.tree[node]
        mid = (start + end) // 2
        L, L_idx = self.queryMinVal(start, mid, 2*node+1, left, right)
        R, R_idx = self.queryMinVal(mid+1, end, 2*node+2, left, right)
        if L < R:
            return L, L_idx
        return R, R_idx

class Solution:
    def minNumberOperations(self, target: List[int]) -> int:
        def dfs(start, end, base):
            nonlocal ret
            if start > end or start < 0 or end >= len(target):
                return 0
            min_val, min_pos = seg_tree.queryMinVal(0, len(target)-1, 0, start, end)
            ret += min_val - base
            dfs(start, min_pos-1, min_val)
            dfs(min_pos+1, end, min_val)
        
        seg_tree = SegmentTree(target)
        ret = 0 
        dfs(0, len(target)-1, 0)
        return ret

In [54]:
solution = Solution()
solution.minNumberOperations(target = [1,1,1,1])

1

In [ ]:
[1,2,3,2,1]
0-4: min_val = 1, min_pos = 0 ret += min_val, base = 1
1-4：min_val = 1, base = 1, ret += min_val-base
1-3: min_val = 2, base = 1, ret += min_val - base 

In [ ]:
from typing import List

class Solution:
    def minNumberOperations(self, target: List[int]) -> int:
        cur = 0
        ret = 0
        for v in target:
            if v > cur:
                ret += v - cur
            cur = v
        return ret